In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [3]:
warnings.filterwarnings("ignore",".*GUI is implemented.*")

In [4]:
from importlib import reload
import lib2

In [5]:
from time import sleep

In [6]:
from lib import data_management as dm

In [7]:
import drivers

In [8]:
from lib2.IQPulseSequence import *

In [9]:
from drivers.KeysightAWG import *

In [10]:
if_freq = 50e6

calibrations = (dm.load_IQMX_calibration_database("TEST", -6))
ro_cal = calibrations.get(frozenset(dict(lo_power=0, ssb_power=-30, lo_frequency=7.047e9, 
                                      if_frequency=0, waveform_resolution=1).items()))
ro_pb = PulseBuilder(ro_cal)

calibrations = (dm.load_IQMX_calibration_database("TESTQ", -6))
q_cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-10, lo_frequency=7.9e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=0.1).items()))
q_pb = PulseBuilder(q_cal)

In [22]:
ro_awg = KeysightAWG("AWG3")

In [17]:
q_awg = KeysightAWG("AWG2")

In [85]:
from drivers.Keysight_DSOX2014 import *

In [86]:
dso = Keysight_DSOX2014("DSO2014")

# Spectroscopy

## Single-tone spectroscopy

In [7]:
# reload(lib2.Measurement)

In [10]:
# reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

In [97]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100, 5).build())

In [66]:
STS = SingleToneSpectroscopy("VI-anticrossing", "Xmon Al BMSTU S444 2", vna = 'vna2', src = 'yok3')
vna_parameters = {"bandwidth":500, "freq_limits":(7.04e9, 7.06e9), "nop":201, "power":-20, "averages":1}
STS.set_fixed_parameters(vna_parameters)
currents = linspace(-0.9e-3, 0.75e-3, 100)
STS._src.set_current(currents[0])
sleep(1)
STS.set_swept_parameters({'current': (STS._src.set_current, currents)})
result = STS.launch()
STS._src.set_current(0);

vna2 is already initialized
yok3 is already initialized
Started at:  2017-05-10 19:40:54.585609


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, current: 7.50e-04]: , average cycle time: 0.47 s          

In [65]:
result.visualize()

In [44]:
result.save()

## Two-tone spectroscopy

In [100]:
# reload(lib2.Measurement)
reload(lib2.TwoToneSpectroscopyBase)
reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [98]:
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100,5).build())

#### Qubit spectrum

In [14]:
Measurement._actual_devices.keys()

dict_keys(['vna3', 'yok4'])

In [101]:
TTS = FluxTwoToneSpectroscopy("VI-two-tone", "Xmon Al BMSTU S444 2", vna_name="vna2", 
                              mw_src_name="mxg", current_src_name="yok3")
vna_parameters = {"bandwidth":50, "freq_limits":(7.04e9, 7.06e9), "nop":1, "power":-20, "averages":1}
mw_src_parameters = {"power":-20}
mw_src_frequencies = linspace(7.88e9, 7.91e9, 50)
currents = linspace(-0.3e-3, 0.05e-3, 50)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7060000000.0)                    at qubit's sweet spot (-0.12 mA)
Detected frequency is 7.04724 GHz, at 0.67 mU and 4.87 degrees


In [102]:
result = TTS.launch()

Started at:  2017-05-19 11:34:08.852049
Time left: 0 h 1 m 1.02 s, parameter value: -7.857e-05, average cycle time: 0.07 s           

In [104]:
result.visualize();

In [28]:
result.save()

In [20]:
result._name = "5.5GHz-two-tone-w-loop_-0.5mA-and-coil"

In [46]:
TTS._mw_src.set_output_state("OFF")

#### Linewidth from excitation power

In [185]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [183]:
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":0, "averages":1}
mw_src_parameters = {}
mw_src_frequencies = linspace(7.88e9, 7.92e9,200)
mw_src_powers = linspace(-20, 15, 21)
current = -0.11e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04716 GHz, at 0.10 mU and 33.83 degrees


In [187]:
result = PTS.launch()

Started at:  2017-05-10 21:15:08.685005
Time left: 0 h 10 m 10.55 s, parameter value: -2.000e+01, average cycle time: 0.15 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 9 m 25.8 s, parameter value: -2.000e+01, average cycle time: 0.14 s           

In [ ]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [106]:
result.visualize()

In [103]:
result.save()

In [ ]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [ ]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

#### AC Stark

In [206]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [208]:
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":1000, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-10, "averages":100}
mw_src_parameters = {"power":-15, "frequency":-1}
mw_src_frequencies = linspace(7.85e9, 7.9e9, 100)
vna_powers = linspace(-30, 0, 31)
current = -0.11e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04716 GHz, at 0.25 mU and 33.57 degrees


In [21]:
result = ASTS.launch()

Started at:  2017-04-25 22:15:13.034443
Time left: 0 h 27 m 19.46 s, parameter value: -3.000e+01, average cycle time: 0.53 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 0.000e+00, average cycle time: 0.16 s              

In [112]:
result.visualize()

In [12]:
result.save()

In [88]:
TTS.close_devs("mxg")

__________________________

# Time-domain measurements

#### Check mixers:

In [44]:
q_lo = TTS._mw_src
q_lo.set_output_state("ON")
q_lo.set_frequency(7.94e9)
q_lo.set_power(15)

In [56]:
q_lo.visa_instr.close()

In [49]:
TTS._vna.set_nop(1)
TTS._vna.set_power(0)
TTS._vna.set_center(7.04720e9)
TTS._vna.sweep_continuous()

In [60]:
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(20).build())
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(20).build())

In [61]:
q_awg.output_pulse_sequence(q_pb.add_sine_pulse(20,0).build())
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(20).build())

In [52]:
print(ro_cal)

Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7047000000.0, 'lo_power': 0, 'if_frequency': 0, 'ssb_power': -30, 'waveform_resolution': 1}
Optimization results: {'dc': -91.583229064941406, 'dc_open': [-29.993877410888672]}
Optimization parameters {'dc_offsets': array([ 0.00531541, -0.06928231]), 'dc_offsets_open': array([ 0.33584961,  0.33543945]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 0 m 17.55 s
Finished at: 2017-05-19 10:45:38.947612


### Set qubit frequency

In [105]:
TTS._current_src.set_current(-0.111e-3)

True

### Rabi 

In [11]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRabiOscillations)
from lib2.DispersiveRabiOscillations import DispersiveRabiOscillations

In [12]:
DRO = DispersiveRabiOscillations("VI-rabi", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', )
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
exc_frequency = 7.89919e9
excitation_durations = linspace(1, 350, 38)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":8000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRO.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DRO.set_swept_parameters(excitation_durations)

The device vna2 is detected as PNA-L2
The device awg3 is detected as AWG3
The device awg2 is detected as AWG2
The device mxg is detected as MXG
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.03 mU and 21.46 degrees


In [13]:
result = DRO.launch()

Started at:  2017-05-19 14:06:30.282909
Time left: 0 h 1 m 37.95 s, [excitation_duration: 3.87e+01]: , average cycle time: 2.97 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [excitation_duration: 3.50e+02]: , average cycle time: 2.98 s         

In [110]:
result.visualize();

In [107]:
result.save()

In [43]:
from lib2.MeasurementResult import *
result = MeasurementResult.load("Xmon Al BMSTU S444 2","VI-ramsey")

In [44]:
result.visualize()

(<matplotlib.figure.Figure at 0x9dfea58>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000001180D9B0>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000009D7C4A8>], dtype=object),
 (None, None))

In [15]:
pi_pulse_duration = result.get_pi_pulse_duration()*1e3
pi_pulse_duration

68.414655278229148

In [16]:
pi_pulse_duration = 68.414655278229148

### Decay

In [17]:
# reload(lib2.VNATimeResolvedMeasurement)
# reload(lib2.DispersiveDecay)
from lib2.DispersiveDecay import *

In [31]:
DD = DispersiveDecay("VI-decay", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', )
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
q_frequency = 7.89939e9
readout_delays = linspace(0, 3000, 100)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "pi_pulse_duration":pi_pulse_duration}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DD.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, rabi_sequence_parameters)
DD.set_swept_parameters(readout_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.04 mU and 9.98 degrees


In [32]:
dd_result = DD.launch()

Started at:  2017-05-14 12:53:53.472372
Time left: 0 h 3 m 43.43 s, [readout_delay: 7.27e+02]: , average cycle time: 2.98 s       

In [115]:
dd_result.visualize();

(<matplotlib.figure.Figure at 0x14fafe48>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000001524CCF8>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000015EC2630>], dtype=object),
 (None, None))

In [116]:
dd_result.save()

### Ramsey

In [18]:
# reload(lib2.IQPulseSequence)
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRamsey)
from lib2.DispersiveRamsey import *

In [19]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                                                  q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_delays = linspace(0, 2000, 100)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency-5e6, rabi_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.03 mU and 24.02 degrees


In [20]:
dr_result = DR.launch()

Started at:  2017-05-19 14:08:50.986957
Time left: 0 h 4 m 43.63 s, [ramsey_delay: 8.08e+01]: , average cycle time: 2.99 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [ramsey_delay: 2.00e+03]: , average cycle time: 2.99 s         

In [21]:
dr_result.visualize();

In [22]:
dr_result.save()

### Echo

In [145]:
# reload(lib2.VNATimeResolvedMeasurement)
# from lib2.VNATimeResolvedMeasurement import *
reload(lib2.DispersiveHahnEcho)
from lib2.DispersiveHahnEcho import *

In [149]:
DHN = DispersiveHahnEcho("VI-hahn-echo", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":50, "averages":2}
current = -0.11e-3
q_frequency = 7.89939e9
echo_delays = linspace(10, 7000, 100)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
DHN.set_fixed_parameters(vna_parameters, ro_cal, q_cal, q_frequency, current, rabi_sequence_parameters)
DHN.set_swept_parameters(echo_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04710 GHz, at 0.04 mU and 15.04 degrees


In [150]:
dhn_result = DHN.launch()

Started at:  2017-05-12 20:43:45.974751
Time left: 0 h 0 m 0.0 s, [echo_delay: 7.00e+03]: , average cycle time: 10.3 s           

In [148]:
dhn_result.visualize();

In [151]:
dhn_result.save()

In [32]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-hahn-echo").visualize()

(<matplotlib.figure.Figure at 0xd808da0>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000B1F7320>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000011049AC8>], dtype=object),
 (None, None))

### Ramsey fringes

In [69]:
# reload(lib2.Measurement)
# reload(drivers.KeysightAWG)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
reload(lib2.DispersiveRamseyFringes)
from lib2.DispersiveRamseyFringes import *

In [65]:
DRF = DispersiveRamseyFringes("VI-ramsey-fringes", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "averages":2}
q_frequency = 7.89939e9
excitation_freqs = linspace(q_frequency-10e6, q_frequency+10e6, 200)
ramsey_delays = linspace(10, 5000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRF.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRF.set_swept_parameters(ramsey_delays, excitation_freqs)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.03 mU and 7.84 degrees


In [66]:
drf_result = DRF.launch()

Started at:  2017-05-13 10:36:36.093113
Time left: 0 h 0 m 0.0 s, [ramsey_delay: 5.00e+03, excitation_frequency: 7.96e+09]: , average cycle time: 0.24 s           

In [67]:
drf_result.visualize()

(<matplotlib.figure.Figure at 0x48ba0e48>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x0000000050AFB860>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000001A82A550>], dtype=object),
  <matplotlib.axes._axes.Axes at 0x50bf0390>])

In [68]:
drf_result.save()

In [70]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-ramsey-fringes").visualize()

0 data/Xmon Al BMSTU S444 2\May 13 2017\09-53-46 - VI-ramsey-fringes\VI-ramsey-fringes.pkl
1 data/Xmon Al BMSTU S444 2\May 13 2017\10-36-36 - VI-ramsey-fringes\VI-ramsey-fringes.pkl
More than one file found. Enter an index from listed above:
1


(<matplotlib.figure.Figure at 0x57b1d668>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x0000000057B79B70>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000057792E80>], dtype=object),
  <matplotlib.axes._axes.Axes at 0x57930668>])

### Rabi shevrons

In [103]:
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
reload(lib2.DispersiveRabiChevrons)
from lib2.DispersiveRabiChevrons import *

In [104]:
DRC = DispersiveRabiChevrons("VI-rabi-shevrons", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "averages":2}
q_frequency = 7.89939e9
excitation_freqs = linspace(q_frequency-10e6, q_frequency+10e6, 200)
ramsey_delays = linspace(10, 5000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRC.set_swept_parameters(ramsey_delays, excitation_freqs)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.03 mU and 8.89 degrees


In [105]:
drc_result = DRC.launch()

Started at:  2017-05-13 15:27:30.841761
Time left: 0 h 0 m 0.0 s, [excitation_duration: 5.00e+03, excitation_frequency: 7.91e+09]: , average cycle time: 0.24 s           

In [106]:
drc_result.visualize();

In [108]:
drc_result.save()

### Dispersive shift measurement

In [75]:
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
reload(lib2.TimeResolvedDispersiveShiftSpectroscopy)
from lib2.TimeResolvedDispersiveShiftSpectroscopy import *

In [94]:
DSS = TimeResolvedDispersiveShiftSpectroscopy("VI-rabi-dispersive-shift-exact-q-freq", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":100, "averages":2}
exc_frequency = 7.89919e9
excitation_durations = linspace(1, 3000, 100)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DSS.set_fixed_parameters(vna_parameters, q_frequency, ro_awg_params, q_awg_params, sequence_parameters)
DSS.set_swept_parameters(excitation_durations)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized


In [95]:
dss_result = DSS.launch()

Started at:  2017-05-14 22:48:05.121530
Time left: 0 h 0 m 0.0 s, [excitation_duration: 3.00e+03]: , average cycle time: 19.14 s          

In [97]:
dss_result.save()

In [76]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-rabi-dispersive-shift").visualize();

In [96]:
dss_result.visualize();

(<matplotlib.figure.Figure at 0x9f0f0b8>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000D2A8208>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000000F1CE160>], dtype=object),
  <matplotlib.axes._axes.Axes at 0xe73a6a0>])

______________________________________________

## Pulse optimization

### Angular Ramsey ($0^{th}$ order APE) 

In [2]:
from lib2.DispersiveAngularRamsey import *

In [ ]:
DAR = DispersiveAngularRamsey("VI-angular-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                            q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_angles = linspace(0, 2*pi, 100)
APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                    "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DR.set_swept_parameters(ramsey_angles)